# Stock Check Process

In [1]:
import kafka

import time

import numpy as np

import json

import pprint

from IPython.display import clear_output


## Read the stock_check_queue; perform a stock check (we are out of 8 GiB RAM DIMMs, everything else we have in stock); for in stock items, send them to the fulfillment_queue;  for out of stock items, send them to the back_order_queue

In [2]:
producer = kafka.KafkaProducer(bootstrap_servers=['kafka:29092'])


In [3]:
topic = "stock_check_queue"

stock_check_queue = kafka.KafkaConsumer(topic, 
                                       bootstrap_servers=['kafka:29092'], 
                                       auto_offset_reset='earliest')


In [9]:
for message in stock_check_queue:
    
    clear_output(wait=True)
    
    message_dict = json.loads(message.value)
    
    print("\n=================================")
    print("   Stock Check Process")
    print("=================================\n")
    print("---------------------------------------------")
    print("   Offset:", message.offset, "   Order ID:", message_dict["order_id"])
    print("---------------------------------------------")
    
    fulfillment_list = []
    
    back_order_list = []
    
    for item in message_dict["line_items"]:
        
        if item["product_id"] == 2:
            
            back_order_list.append(item)
            
        else:
            
            fulfillment_list.append(item)
    
    if len(fulfillment_list) > 0:
        
        
        sub_total = 0
        
        for f in fulfillment_list:
            sub_total += f["total"]
            
        sub_total = round(sub_total, 2)
        
        tax = round(sub_total * 0.08, 2)
        
        total = round(sub_total + tax, 2)
             
        fulfillment = { "order_id": message_dict["order_id"],
                        "sub_total": sub_total,
                        "tax": tax,
                        "total": total,
                        "line_items": fulfillment_list}
        print("\nIn stock items:\n")
        pprint.pprint(fulfillment, sort_dicts=False)
        
        producer.send("fulfillment_queue", json.dumps(fulfillment).encode())
        producer.send("stock_check_pub_sub", json.dumps(back_order).encode())
        producer.send("stock_check_stream", json.dumps(back_order).encode())
    
    if len(back_order_list) > 0:
        
        sub_total = 0
        
        for b in back_order_list:
            sub_total += b["total"]
            
        sub_total = round(sub_total, 2)
        
        tax = round(sub_total * 0.08, 2)
        
        total = round(sub_total + tax, 2)
        
        back_order = { "order_id": message_dict["order_id"],
                       "sub_total": sub_total,
                       "tax": tax,
                       "total": total,
                       "line_items": back_order_list}
        print("\nBack Order Items:\n")
        pprint.pprint(back_order, sort_dicts=False)
        
        producer.send("back_order_queue", json.dumps(back_order).encode())
    
    time.sleep(2.0)


   Stock Check Process

---------------------------------------------
   Offset: 1071    Order ID: 15446
---------------------------------------------

In stock items:

{'order_id': 15446,
 'sub_total': 2878.65,
 'tax': 230.29,
 'total': 3108.94,
 'line_items': [{'product_id': 1,
                 'product_name': 'ARM Board',
                 'product_price': 99.95,
                 'quantity': 9,
                 'total': 899.55},
                {'product_id': 3,
                 'product_name': '16 GiB RAM DIMM',
                 'product_price': 69.95,
                 'quantity': 9,
                 'total': 629.55},
                {'product_id': 4,
                 'product_name': '32 GiB RAM DIMM',
                 'product_price': 149.95,
                 'quantity': 9,
                 'total': 1349.55}]}

Back Order Items:

{'order_id': 15446,
 'sub_total': 329.45,
 'tax': 26.36,
 'total': 355.81,
 'line_items': [{'product_id': 2,
                 'product_name': '8 GiB RAM 

KeyboardInterrupt: 